Define functions for plotting benchmarking results

In [18]:
import inspect
import itertools
import numpy as np
import pandas as pd
import pygmo as pg
import sys

from numba import jit

%run Test_functions.ipynb

### Showcase on how to use the Pygmo library

In [6]:
# Set some parameters first
int_dim_rosenbrock = 5
int_gen = 1000
int_pop_size = 200
int_seed = 209311
int_verbosity = 1
list_algo_name = ["cmaes", "gaco", "de"]

# Example problems can be called here. Can define UDPs too.
# A problem needs to be constructed with pg.problem
prob_rosenbrock = pg.problem(pg.rosenbrock(int_dim_rosenbrock))
vec_arg_rosenbrock = pg.rosenbrock(int_dim_rosenbrock).best_known()
#float_target_rosenbrock = pg.rosenbro

# Most used methods are fitness and get_(f/g/h)evals. Can set a seed.

# User defined algorithms are provided by the algorithm type
algo_cmaes = pg.algorithm(pg.cmaes(int_gen, seed = int_seed))
algo_gaco = pg.algorithm(pg.gaco(int_gen, seed = int_seed))
algo_de = pg.algorithm(pg.de(int_gen, seed = int_seed))

# To benchmark, we need to set a value > 0 for verbosity to generate a log file
algo_cmaes.set_verbosity(int_verbosity)
algo_gaco.set_verbosity(int_verbosity)
algo_de.set_verbosity(int_verbosity)

# The population is a pool of candidate solution to the problem
pop_rosenbrock = pg.population(prob_rosenbrock, size = 100, seed = int_seed)

# Can get the number of function evaluations by using
pop_rosenbrock.problem.get_fevals

# Islands are used to allow multithread computation
# size gives the number of function evaluations
isl_rosenbrock = pg.island(algo = algo_cmaes, prob = prob_rosenbrock, size = 20)

# Archipelage is used for parallelization and allow for migration of solutions. 
# Might be useful as a special case and how this can help.

# Solve rosenbrock using three algorithms
pop_rosenbrock_cmaes = algo_cmaes.evolve(pop_rosenbrock)
pop_rosenbrock_gaco = algo_gaco.evolve(pop_rosenbrock)
pop_rosenbrock_de = algo_de.evolve(pop_rosenbrock)

# Get the log files as a pandas dataframe
log_cmaes_rosenbrock = algo_cmaes.extract(pg.cmaes).get_log()
log_gaco_rosenbrock = algo_gaco.extract(pg.gaco).get_log()
log_de_rosenbrock = algo_de.extract(pg.de).get_log()

# Extract when 

### Questions

- How to take into account, that parameter tuning was used?
    - Could start from default settings. (x)
    - For every algo get a random grid and check whether the stopping condition is met for such methods.
    - Also vary population size. (x)
- Since starting points are random should repeat this many times. Can implement the same seed for all algos. (x)
- How to find out, when the stopping criterion was reached?
    - Lookup log file and find the number of function iterations. (x)
- Would also like to show how the population size affects performance. (x)
- Maximum accuracy measure $ M $ should be equal to the tolerance considered. This should happen after all optimizers have been tuned.

In [43]:
def acc(population, x0, f0, target, f_tol = 1e-06):
    
    id_best = population.best_idx()
    f_best = population.champion_f
    x_best = population.champion_x
    
    # Target is a list supplied by the user
    f_prime = target[1]
    x_prime = target[0]
    
    # Calculate accuracy measures
    f_acc_n = (f_best - f_prime) / (f0 - f_prime)
    x_acc_n = np.linalg.norm(x_best - x_prime) / np.linalg.norm(x0 - x_prime)
    f_acc_l = -np.log10(f_acc_n)
    x_acc_l = -np.log10(x_acc_n)
    #breakpoint()
    # Boolean for convergence. Use stopping criteria from pygmo library
    converged = (np.abs(f_best - f_prime) < f_tol)
    
    # Returns a series
    list_acc_index = ["acc_norm_f", "acc_norm_x", "acc_log_f", "acc_log_x", "converged"]
    return pd.Series([f_acc_n, x_acc_n, f_acc_l, x_acc_l, converged], 
                     index = list_acc_index,
                     dtype = float
                    )

def get_results_algo_problem(problem, algorithm_name, target, kwargs_algorithm = None, 
                             gen = 1000, pop_size = 100, iterations = 100, 
                             seed = 2093111, verbosity = 1):
    
    # Get a sequence of seeds for reproducability
    array_seeds = np.arange(start = seed, stop = seed + iterations)
        
    # Empty lists to store logs and evals
    list_acc = list()
    list_evals = list()
    list_logs = list()
    
    # Now loop over
    for iter in range(iterations):
        
        # Generate a population that with size equal pop_size
        population = pg.population(problem, size = pop_size, seed = seed + iter)
        
        # Multiple starting points. Use best x0 at the starting point.
        x0 = population.champion_x
        f0 = population.champion_f
        
        #breakpoint()
        if kwargs_algorithm is None:
            algorithm = pg.algorithm(getattr(pg, algorithm_name)(gen, seed = seed + iter))
        if algorithm_name == "mbh":
            algorithm = pg.algorithm(getattr(pg, algorithm_name)(seed = seed + iter, **kwargs_algorithm))
        else:
            algorithm = pg.algorithm(getattr(pg, algorithm_name)(gen, seed = seed + iter, **kwargs_algorithm))
        algorithm.set_verbosity(verbosity)
        population = algorithm.evolve(population)
        log = algorithm.extract(getattr(pg, algorithm_name)).get_log()
        #breakpoint()
        # Performance profiles need all observations
        df_log = pd.DataFrame(log).iloc[:, 0:3]
        df_log.columns = ["gen", "f_evals", "best"]
        
        # Add column for iteration number and algorithm
        df_log["iteration"] = iter + 1
        df_log["algorithm"] = algorithm_name
        
        # A column for relative loss log and negative log_10 loss
        # Since the log file, doesn't return the best x vector can only compute function loss
        df_log["acc_norm_f"] = (df_log["best"] - target[1]) / (f0 - target[1])
        df_log["acc_log_f"] = -np.log(df_log["acc_norm_f"])
        
        # Append results to lists
        ser_acc = acc(population, x0, f0, target)
        ser_acc["iteration"] = iter + 1
        ser_acc["algorithm"] = algorithm_name
        list_acc.append(ser_acc)
        list_evals.append([
            population.problem.get_fevals(),
            population.problem.get_gevals(),
            population.problem.get_hevals()
        ])
        list_logs.append(df_log)
        
    # Put everything together
    #breakpoint()
    df_acc = pd.DataFrame(list_acc)
    df_acc = df_acc.astype({"converged": bool})
    df_evals = pd.DataFrame(list_evals, columns = ["f_eval", "g_eval", "h_eval"])
    df_evals["iteration"] = np.arange(1, iterations + 1)
    df_evals["algorithm"] = algorithm_name
    df_logs = pd.concat(list_logs)
        
    return df_acc, df_evals, df_logs

In [44]:
# Check accuracy
target = [pg.rosenbrock(dim = 5).best_known()]
target.append(prob_rosenbrock.fitness(target[0]))
x0 = pop_rosenbrock.champion_x
f0 = pop_rosenbrock.champion_f
acc(pop_rosenbrock_cmaes, x0, f0, target)

acc_norm_f    9.958784e-13
acc_norm_x    1.137660e-05
acc_log_f     1.200179e+01
acc_log_x     4.943988e+00
converged     1.000000e+00
dtype: float64

In [45]:
# Check whether the function works
problem_test = pg.problem(pg.rosenbrock(dim = 5))
algorithm_name_test = "mbh"
kwargs_algorithm = {"algo": pg.nlopt("lbfgs")}
target = [pg.rosenbrock(dim = 5).best_known()]
target.append(prob_rosenbrock.fitness(target[0]))

get_results_algo_problem(
    problem = problem_test,
    algorithm_name = algorithm_name_test,
    target = target,
    kwargs_algorithm = kwargs_algorithm
)

C:\Users\Wilms\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(      acc_norm_f    acc_norm_x  acc_log_f  acc_log_x  converged  iteration  \
 0   2.005449e-26  1.511553e-12  25.697788  11.820577       True        1.0   
 1   1.046271e-25  1.525250e-12  24.980356  11.816659       True        2.0   
 2   3.600143e-27  1.803605e-12  26.443680  11.743859       True        3.0   
 3   4.778339e-03  5.229962e-01   2.320723   0.281501      False        4.0   
 4   4.029363e-30  1.034359e-14  29.394764  13.985329       True        5.0   
 ..           ...           ...        ...        ...        ...        ...   
 95  4.785973e-03  8.336453e-01   2.320030   0.079019      False       96.0   
 96  1.706487e-27  1.414891e-12  26.767897  11.849277       True       97.0   
 97  3.248842e-26  4.355849e-13  25.488271  12.360927       True       98.0   
 98  3.322163e-03  2.149745e-01   2.478579   0.667613      False       99.0   
 99  6.374234e-28  2.302316e-13  27.195572  12.637835       True      100.0   
 
    algorithm  
 0        mbh  
 1        mbh  
 2

This code block runs the same algorithm on a group of problems for differing population sizes

In [12]:
def get_results_popsize(problem, algorithm_name, target, kwargs_algorithm = None,
                        gen = 1000, list_pop_size = [20, 50, 100, 250], iterations = 100, 
                        seed = 2093111, verbosity = 1):
    
    # For different population sizes apply get_results_algo_problem
    # Store as a dataframe
    # Problem is again defined outside this function
    list_acc_popsize = list()
    list_evals_popsize = list()
    list_logs_popsize = list()
    
    for pop_size in list_pop_size:
        
        # Store in differing
        df_acc, df_evals, df_logs = get_results_algo_problem(
            problem,
            algorithm_name,
            target,
            kwargs_algorithm,
            gen,
            pop_size,
            iterations,
            seed,
            verbosity
        )
        
        # Set popsize as variable
        df_acc["pop_size"] = pop_size
        df_evals["pop_size"] = pop_size
        df_logs["pop_size"] = pop_size
        
        list_acc_popsize.append(df_acc)
        list_evals_popsize.append(df_evals)
        list_logs_popsize.append(df_logs)
    
    # Put together into one big dataframe
    df_acc_popsize = pd.concat(list_acc_popsize)
    df_evals_popsize = pd.concat(list_evals_popsize)
    df_logs_popsize = pd.concat(list_logs_popsize)
    
    return df_acc_popsize, df_evals_popsize, df_logs_popsize

In [14]:
get_results_popsize(prob_rosenbrock, "sea", target)

(    acc_norm_f  acc_norm_x  acc_log_f  acc_log_x  converged  iteration  \
 0     0.001441    0.547146   2.841466   0.261897      False        1.0   
 1     0.000293    0.301309   3.533351   0.520989      False        2.0   
 2     0.000478    0.414025   3.320418   0.382973      False        3.0   
 3     0.001540    2.089224   2.812557  -0.319985      False        4.0   
 4     0.000965    0.070802   3.015428   1.149955      False        5.0   
 ..         ...         ...        ...        ...        ...        ...   
 95    0.017226    0.170188   1.763824   0.769072      False       96.0   
 96    0.000137    0.064716   3.863303   1.188987      False       97.0   
 97    0.003037    1.036564   2.517561  -0.015596      False       98.0   
 98    0.003439    0.345289   2.463544   0.461817      False       99.0   
 99    0.003530    0.214277   2.452203   0.669024      False      100.0   
 
    algorithm  pop_size  
 0        sea        20  
 1        sea        20  
 2        sea       

Get measures for differing problems and algorithms. Use a for loop again

In [29]:
def get_results_all(
    list_problem_names,
    list_algorithm_names,
    kwargs_problem = None,
    kwargs_algorithm = None,
    gen = 1000,
    list_pop_size = [20, 50, 100, 250],
    iterations = 100,
    seed = 2093111,
    verbosity = 1
):
    
    # Kwargs for problem and algo are a dictionary of dictionaries.
    # The Keys are the problem/algortihm name respectively
    if kwargs_problem is None:
        kwargs_problem = dict()
        for problem_name in list_problem_names:
            kwargs_problem[problem_name] = None
        
    if kwargs_algorithm is None:
        kwargs_algorithm = dict()
        for algorithm_name in list_algorithm_names:
            kwargs_algorithm[algorithm_name] = None
            
    # Check that a kwarg is present for every problem/algorithm
    if len(list_problem_names) != len(kwargs_problem):
        raise KeyError("kwargs_problem has to contain an entry for every problem. If you don't intend \
                       to supply any values set it equal to None")
        
    if len(list_algorithm_names) != len(kwargs_algorithm):
        raise KeyError("kwargs_algorithm has to contain an entry for every algorithm. If you don't intend \
                       to supply any values set it equal to None")
    
    # Containers for output
    list_acc_all = list()
    list_evals_all = list()
    list_logs_all = list()
    
    # Greate a grid of problem and algorithm names over which to loop
    for problem_name, algorithm_name in itertools.product(list_problem_names, list_algorithm_names):
        
        # Some algorithms can't handle 
        # Define the problem
        problem = pg.problem(getattr(pg, problem_name)(**kwargs_problem[problem_name]))
        
        # Get target for given problem
        target = get_target(problem_name, kwargs_problem[problem_name])
        # breakpoint()
        # Apply the get_results_popsize
        df_acc_problem_algorithm, df_evals_problem_algorithm, df_logs_problem_algorithm = \
        get_results_popsize(
            problem,
            algorithm_name,
            target,
            kwargs_algorithm[algorithm_name],
            gen,
            list_pop_size,
            iterations,
            seed,
            verbosity
        )
        
        # Add a column to every dataframe identifying the problem considered
        df_acc_problem_algorithm["problem"] = problem_name
        df_evals_problem_algorithm["problem"] = problem_name
        df_logs_problem_algorithm["problem"] = problem_name
        
        # Put into list containers
        list_acc_all.append(df_acc_problem_algorithm)
        list_evals_all.append(df_evals_problem_algorithm)
        list_logs_all.append(df_logs_problem_algorithm)
        
    # Put into dataframe format
    df_acc_all = pd.concat(list_acc_all)
    df_evals_all = pd.concat(list_evals_all)
    df_logs_all = pd.concat(list_logs_all)
    
    return df_acc_all, df_evals_all, df_logs_all

In [33]:
list_problem_names = ["rosenbrock", "rastrigin", "schwefel", "ackley"]
list_algorithm_names = ["de", "cmaes", "mbh"]
kwargs_problem = {"rosenbrock": {"dim": 5}, "rastrigin": {"dim": 3}, "schwefel": {"dim": 2}, "ackley": {"dim": 10}}
kwargs_algorithm = {"de": None, "cmaes": None, "mbh": {"algo": pg.nlopt("lbfgs")}}

get_results_all(
    list_problem_names,
    list_algorithm_names,
    kwargs_problem,
    kwargs_algorithm
)

TypeError: _mbh_init() got multiple values for argument 'algo'

### Setup

Need to get some solved optimization algorithms first. Test with the Ackley function.

In [47]:
list_problem_names = ["rosenbrock", "rastrigin","schwefel", "ackley", "griewank", 
                      "lennard_jones", "luksan_vlcek1", "cec2014",
                     "cec2013", "cec2006"]
list_algorithm_names = ["bee_colony", "de", "sea", "sga", "sade", "cmaes", 
                        "pso", "pso_gen", "mbh"]
kwargs_problem = {"rosenbrock": {"dim": 5}, "rastrigin": {"dim": 10}, "schwefel": {"dim": 2},
                 "ackley": {"dim": 4}, "griewank": {"dim": 6}, "lennard_jones": {"atoms": 30},
                 "luksan_vlcek1": {"dim": 7}, "cec2014": {"prob_id": 1, "dim": 8},
                 "cec2013": {"prob_id": 2, "dim": 3}, "cec2006": {"prob_id": 3}}

### Trajectory Plot

A trajectory plot shows how the algorithm moved from iteration to iteration. Want to have one plot for multiple algorithms. EA need a single plot with multiple points. Since all algos start from multiple points it could be wise to have an animated plot

### Convergence Plot

Plots the best function value against the number of function evaluations. An average runtime plot could be useful. For example how it changes with 

### Performance profiles
For a given set of problems $ \mathcal{P} $, solvers $ \mathcal{S} $ and a convergence test $ \mathcal{T} $. They are a performance measre $ t_{p, s} > 0 $, where $ p,\, s $ are indices for a given problem and solver $ (p, s) \in \mathcal{P} \times \mathcal{S} $. Now the value $ r_{p, s} $ is defined as:

$$ r_{p, s} = \begin{cases} 
\frac{t_{p, s}}{\min\{t_{p, s}: \, s \in \mathcal{S} \}} \quad \text{if convergence is passed} \\
\infty \quad \text{if convergence fails}
\end{cases} $$

The best performing optimizer will have $ r_{p, s} = 1 $. For a specific problem $ p $ and cutoff $ \tau > 1 $, the performance profile for solver $ s $ is defined as follows:

$$ \rho_s(\tau) = \frac{1}{|\mathcal{P}|} \text{size} \{p \in \mathcal{P}: \, r_{p, s} \leq \tau \} $$

where $ |\mathcal{P} | $ is the cardinality of the set $ \mathcal{P} $. $ \rho_s(1) $ represents the fraction that solver $ s $ is the best performing solver. Want to identify solvers with high values. Remember that performance profiles depend of the solvers considered. For comparing only two optimizers a new profile should be drawn.

Important is what measure we consider for performance. Possible are function value or distance to optimal solution. For measuring performance by function value

$$ m_{p, s} = \frac{\hat{f}_{p, s}(k) - f^*}{f_w - f^*} $$

where $ k $ is the value of function evaluations and $ f_w $ is the worst value after $ k $ evaluations.

Performance profiles can allow to assess both speed and success rate. However, caution is advised as it is not ceretain that the best solution found is the best.

### Accuracy Profiles
Visualize an entire benchmarking test set. Only applicable for fixed-cost problems. For every $ s \in \mathcal{S} $ and $ p \in \mathcal{P} $, we calculate an accuracy measure is calculated, where $ M $ is a maximum improvement value. 

$$ \gamma_{p, s} = \begin{cases}
-f_{acc}^{p, s}, \quad \text{if } -f_{acc}^{p, s} \leq M \\
M, \quad -f_{acc}^{p, s} > M \text{ or } f_{acc}^{p, s} \text{ is undefined}
\end{cases} $$

where $ f_{acc}^{p, s} = \log_{10}(f(\bar{x}_{p, s} - f(x_p^*)) - \log_{10}(f(x_p^0) - f(x_p^*)) $, $ x_{p, s} $ is the candidate solution, obtained by solver $ s $ for problem $ p $, $ x_p^* $ is the optimal point for problem $ p $, and $ x_p^0 $ is the initial point for problem $ p $. To measure the performance we calculate 

$$ R_s (\tau) = \frac{1}{|\mathcal{P}|} \text{size} \{ \gamma_{p, s} | \gamma_{p, s} \geq \tau, \, p \in \mathcal{P} \} $$

it shows the proportion of problems for which the solver $ s $ achieves an accuracy of at least $ \tau $. Only suitable for fixed cost data-sets.

### Data Profiles
Proposed for derivative-free optimization algorithms. They show, which percentage of problems for a given value $ \tau $ can be solved within the budget of $ k $ function evaluations. Since it is assumed, that the number of functions evluations grows with the dimension of the problem $ n_p $, it is defined as

$$ d_s (k) = \frac{1}{\mathcal{P}} \text{size} \{p \in \mathcal{P}: \, \frac{t_{p, s}}{n_p + 1} \leq k \}$$


here, $ t_{p, s} $ is the number of function evaluations required to satisfy the convergence test, $ d_s (k) $ is the fraction of problems $ p $ solved by $ s $ within $ k $ evaluations. Data profiles are independent of other solvers. To compare gradient-free and gradient-based methods I will also implement a data profiles that accounts for number of gradient and hessian evaluations.